In [1]:
import cv2
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing import image
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

2024-07-11 19:43:56.437857: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/gianluca/dev/formula1-classification/env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
model = VGG16(weights='imagenet', include_top=False)
features = []
labels = []

In [3]:
for turns_name in os.listdir('data/'): #iterate over turns
    if(os.path.isdir('data/'+turns_name)):
        print(turns_name)
        for driver_name in os.listdir('data/'+turns_name): #iterate over drivers
            if (os.path.isdir('data/'+turns_name+'/'+driver_name)):
                
                for img in os.listdir('data/'+turns_name+'/'+driver_name): #iterate over images
                    if(img != '.DS_Store'):
                        print(img)
                        image = cv2.imread('data/'+turns_name+'/'+driver_name+'/'+img)
                        img_data = np.expand_dims(image, axis=0)
                        img_data = preprocess_input(img_data)
                        vgg16_feature = model.predict(img_data)
                        features.append(vgg16_feature.flatten())
                        labels.append(turns_name)


biassono
lec__1__616.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 535ms/step
lec__1__602.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 322ms/step
lec__1__560.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 306ms/step
lec__1__574.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 301ms/step
lec__1__575.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 306ms/step
lec__1__561.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 308ms/step
lec__1__603.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 311ms/step
lec__1__617.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 311ms/step
lec__1__601.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 312ms/step
lec__1__615.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 315ms/step
lec__1__577.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 321ms/step
lec__1__563.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 325ms/step
lec__1__588.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 434ms/step
lec__1__589.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 322ms/step
lec__1__562.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 325ms/step
lec__1__576.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 348ms/step
lec__1__614.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 327ms/step
lec__1__600.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 330ms/step
l

In [4]:
X = np.array(features)
y = np.array(labels)

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.35, random_state=42)
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [8]:
# Valutiamo il modello
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 1.0


In [11]:
cap = cv2.VideoCapture('media/kimi-processed.mp4')    

In [12]:
frame_count = 0
while cap.isOpened():
	ret, frame = cap.read()
	# if frame is read correctly ret is True
	if (frame_count % 4) == 0:
	    cv2.imshow('frame', frame)
	    img_data = np.expand_dims(frame, axis=0)
	    img_data = preprocess_input(img_data)
	    features = []
	    f = model.predict(img_data)
	    features.append(f.flatten())
	    features = np.array(features)
	    predictions = clf.predict(features)
	    print('turns:')
	    print(label_encoder.inverse_transform(predictions))
	if not ret:
		print("Can't receive frame (stream end?). Exiting ...")
		break
	if cv2.waitKey(1) == ord('q'):
		break
	frame_count+=1 
cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 303ms/step
turns:
['parabolica']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 338ms/step
turns:
['parabolica']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 359ms/step
turns:
['parabolica']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 434ms/step
turns:
['parabolica']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 363ms/step
turns:
['parabolica']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 415ms/step
turns:
['parabolica']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 480ms/step
turns:
['parabolica']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 402ms/step
turns:
['parabolica']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 387ms/step
turns:
['parabolica']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 411ms/step
turns:
['parabolica']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 356ms/step
turns:
['parabolica']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 409ms/step
turns:
['parabolica']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 370ms/step
turns:
['parabolica']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 454ms/step
turns:
['parabolica']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 388ms/step
turns:
['parabolica']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 440ms/step
turns:
['parabolica']
1/1 ━━━━━━━━━━━━━━━━━━━━